In [ ]:
import re

import pandas as pd
import plotly.express as px

from Database import Db

In [ ]:
year1_minutes, meta = Db.load_data(hourly=False, meta=True)
year2_minutes = Db.load_data(hourly=False, year=2)

In [ ]:
year1_minutes["Timestamp"] = pd.to_datetime(year1_minutes["Timestamp"], format="%Y-%m-%d %H:%M:%S%z")

In [ ]:
year1_minutes.head()

In [ ]:
year1_minutes.describe().T

In [ ]:
year1_minutes.columns.difference(year2_minutes.columns)

In [ ]:
year2_minutes.columns.difference(year1_minutes.columns)

In [ ]:
del year2_minutes

In [ ]:
totals = [x for x in year1_minutes.columns if "Total" in x]
totals

In [ ]:
fig = px.scatter(year1_minutes, x="Timestamp", y="Load_KPlugLoadsPowerUsage", color="Load_StatusPlugLoadToaster")
fig.show()

In [ ]:
plugs = meta.loc[(meta['Parameter'] == "Status_OnOff") & (meta["Measurement_Location"] == "Kitchen") & (
    meta["Unnamed: 0"].str.contains("Plug"))]["Unnamed: 0"].tolist()

In [ ]:
kitchenplugs = year1_minutes.loc[0:10000, ["Timestamp", "Load_KPlugLoadsPowerUsage"] + plugs].copy()
del year1_minutes
kitchenplugs["Status"] = kitchenplugs[plugs].sum(axis=1)

In [ ]:
(kitchenplugs[plugs] < 0) | (kitchenplugs[plugs] > 1)

In [ ]:
appliance_dict = {}
for appliance in plugs:
    name = " ".join(re.findall('[A-Z][^A-Z]*', appliance.split("Load")[-1]))
    appliance_switch = kitchenplugs[(kitchenplugs[appliance] != kitchenplugs[appliance].shift(1))][
                           ["Timestamp", appliance]][1:]
    appliance_dict[name] = pd.DataFrame(
        {"Timestamp": [appliance_switch.loc[index, "Timestamp"] for index in appliance_switch.index],
         "Appliance": name,
         "Color": ["green" if appliance_switch.loc[index, appliance] == 1 else "red" for index in
                   appliance_switch.index]})

appliance_dict

In [ ]:
fig = px.scatter(kitchenplugs, x=kitchenplugs["Timestamp"], y="Load_KPlugLoadsPowerUsage", color="Status")

height = 1
for appliance, df in appliance_dict.items():
    for index in df.index:
        x = df.loc[index, "Timestamp"].timestamp() * 1000 - (2 + int(str(df.loc[index, "Timestamp"]).split("-")[-1][
                                                                         1])) * 3_600_000  #Plotly bug work around. Computes unix time with offset
        fig.add_vline(x=x,
                      line_width=3,
                      line_dash="dash",
                      annotation_text=df.loc[index, "Appliance"],
                      line_color=df.loc[index, "Color"],
                      annotation_y=height)
    height -= 0.05

fig.update_layout(
    xaxis={'type': 'date', 'tickformat': '%Y-%m-%d %H:%M:%S'}
)

fig.update_layout(
    xaxis_tickformatstops=[
        dict(dtickrange=[None, 60000], value="%H:%M:%S\n%d %b %y"),
        dict(dtickrange=[60000, 3600000], value="%H:%M\n%d %b %y"),
        dict(dtickrange=[3600000, 86400000], value="%d %b %Y")
    ]
)

fig.show()

In [ ]:
kitchenplugs[kitchenplugs["Load_StatusPlugLoadToasterOven"] == 1][["Timestamp", "Load_StatusPlugLoadToasterOven"]]